# Динамика основных показателей по кварталам и месяцам
Пример расчета показателей телеканалов в разбивке по периодам.

## Описание задачи и условий расчета
- Период: 01.01.2022 - 30.09.2022 по кварталам и месяцам + тотал за период
- Временной интервал: 05:00-29:00
- ЦА: Россия 100+
- Место просмотра: Все места (дом+дача)
- Каналы: ПЕРВЫЙ КАНАЛ									
- Статистики: Share, AvReach%, ATVReach

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# выберем телесети по имени (части имени)
cats.get_tv_net()

# далее в запросах будем использовать id телесетей.

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


In [3]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['100+'])

Запрошены записи: 0 - 2
Всего найдено записей: 2



,id,valueId,name,colName,valueName
0,118,1,Население 100+,Cube100Plus100Minus,100+
1,118,2,Население 100+,Cube100Plus100Minus,100-


## Формирование задания
Зададим условия расчета

In [4]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-01-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = 'cube100Plus100Minus = 1'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId = 1'

# Указываем список статистик для расчета
statistics = ['Share', 'AvReachPer', 'ATVReach']

# Указываем срезы
slices = []

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем различные срезы по периодам

In [5]:
periods = {
    # Вариант срезов для разбивки по месяцам
    "По месяцам": {"slices": ['researchMonth']},
    # Вариант срезов для разбивки по кварталам
    "По кварталам": {"slices": ['researchQuarter']},
    # для тотала срез оставляем пустым
    "Тотал": {"slices": []}
}

## Расчет задания

In [6]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой группы срезов формируем задания и отправляем на расчет
for period, logical in periods.items():
    
    # Формируем задание на расчет
    project_name = f"{period}" 
    slices = logical['slices']
              
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)
df

Отправляем задания на расчет
...
Ждем выполнения
Расчет задач (3) [ = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = ] время расчета: 0:04:59.105837
Расчет завершен, получаем результат
Собираем таблицу
...

,prj_name,researchMonth,Share,ATVReach,AvReachPer,researchQuarter
0,По месяцам,2022-01-01,10.063287,80.444645,27.712378,NaN
1,По месяцам,2022-02-01,11.326683,85.398317,29.514161,NaN
2,По месяцам,2022-03-01,10.752596,79.444185,28.724839,NaN
3,По месяцам,2022-04-01,9.763618,75.784271,26.036578,NaN
4,По месяцам,2022-05-01,9.203081,72.224623,25.060972,NaN
5,По месяцам,2022-06-01,8.440991,67.828841,22.732134,NaN
6,По месяцам,2022-07-01,9.194851,72.618766,22.113503,NaN
7,По месяцам,2022-08-01,8.838105,72.259481,21.460337,NaN
8,По месяцам,2022-09-01,9.123237,73.056844,23.451079,NaN
0,По кварталам,NaN,10.693806,81.687999,28.621669,2022-01-01


Подготовливаем таблицу для сохранения

In [7]:
# Объединяем колонки с датами (NULL значения из одной колонки заполняются из другой)
df['date'] = df['researchMonth'].combine_first(df['researchQuarter'])

# Удаляем лишние колонки
df.drop(columns = ['researchMonth', 'researchQuarter'],axis = 1, inplace=True)

# Меняем порядок столбцов
df_res = df[['prj_name', 'date', 'Share', 'AvReachPer', 'ATVReach']]
df_res

,prj_name,date,Share,AvReachPer,ATVReach
0,По месяцам,2022-01-01,10.063287,27.712378,80.444645
1,По месяцам,2022-02-01,11.326683,29.514161,85.398317
2,По месяцам,2022-03-01,10.752596,28.724839,79.444185
3,По месяцам,2022-04-01,9.763618,26.036578,75.784271
4,По месяцам,2022-05-01,9.203081,25.060972,72.224623
5,По месяцам,2022-06-01,8.440991,22.732134,67.828841
6,По месяцам,2022-07-01,9.194851,22.113503,72.618766
7,По месяцам,2022-08-01,8.838105,21.460337,72.259481
8,По месяцам,2022-09-01,9.123237,23.451079,73.056844
0,По кварталам,2022-01-01,10.693806,28.621669,81.687999


## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [8]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('01_months&quarters'))
df_info = mtask.task_builder.get_report_info()
df_res.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()